# Topic Modeling Walkthrough - 20 newsgroups

This walkthrough will take you through a simple topic modeling task. By the end of the tutorial, you should be able to create a simple machine learning workflow to perform topic modeling for a set of email groups.

## Create a project

To start, we'll quickly create a Squirro project that we can work in. To do this you'll need a running Squirro cluster and a valid API token.

In [ ]:
CLUSTER = ""
TOKEN = ""

# get a client
from squirro_client import SquirroClient
client = SquirroClient(client_id=None, client_secret=None, cluster=CLUSTER)
client.authenticate(refresh_token=TOKEN)

# create a project
PROJECT_ID = client.new_project("Topic Modeling Walkthrough").get("id")
print PROJECT_ID

## Loading data

The next step is to load data in our Squirro instance. We can now run a pre-made Squirro data loader script to insert our data set:

In [ ]:
!{"./load.sh %s %s %s" % (CLUSTER, TOKEN, PROJECT_ID)}

## Examine the dataset

Just as in the classification walkthrough, we first want to get an idea of what is in our dataset, so we print a single item to look at:

In [ ]:
# print a single item
item = client.query(project_id=PROJECT_ID, query='label:*',
                    fields=['body','title','keywords'], count=1)['items'][0]
print u'{label} - {title} - {body}'.format(
    title=item['title'], body=item['body'], label=item['keywords']['label'][0])

As you can see, the items in our dataset are group emails, so they include quite a bit of natural language.

Next we'll look at the dataset as a whole to get an idea of the balance between the newsgroups:

In [ ]:
res = client.query(project_id=PROJECT_ID, query='*', aggregations={'label': {}})
for value in res['aggregations']['label']['label']['values']:
    print u'{label} - {count}'.format(label=value['key'], count=value['value'])

As you can see, our dataset has about 1K samples per label, and only 3 labels. The labels represent the topic of each newsgroup, and here the topics are different enough that topic modeling should be effective.

## Build the model workflow

Now that we have an idea of the data we're dealing with, we can move on to building our topic model. To reiterate the goal, we want to build a model that determines the topics in our dataset automatically, without telling it ahead of time what the categories should be (this is an unsupervised learning task).

The heart of Squirro's Machine Learning Service is our custom natural language processing library libNLP. It is what actually does all the processing. Thus our model workflow is simply a libNLP workflow, which we'll walk through now. (For extended documentation for libNLP, see https://squirro.github.io/nlp/).

The libNLP workflow is simply a JSON file with specifications for individual components required for machine learning, so we start with an empty JSON:

In [ ]:
workflow = {}

### Specify the dataset

The first thing we need to do is tell libNLP on which dataset to operate. We do this by providing Squirro queries to `train`, `test`, and `infer` data sets. `train` is the data we want to train the model on. `test` is the data we'd like to test the model on, and `infer` is the data we'd like to predict on (which is typically unlabeled).

In [ ]:
workflow["dataset"] = {
    "train": {"query_string": "dataset:train"},
    "test": {"query_string": "dataset:test"}
}

Here we have already split our dataset into a training and test set using a `dataset` facet during loading. Notice also that `query_string` can be any Squirro query, making it easy to carve out your samples.

### Specify the analyzer

Next we want to tell libNLP the type of machine learning task we have. That way we can later analyze how well we are doing at this task.

In [ ]:
workflow["analyzer"] = {
    "type": "topic_modeling",
    "label_field": "keywords.label",
    "tag_field": "keywords.topics"
}

Here we said we have a `topic_modeling` task, where the ground-truth label is `label` and the field with our create topics is `topics`.

### Specify the pipeline

Finally we need to tell libNLP the steps we'll use to go from unstructured text to a prediction for each item. We do so by defining a pipeline compose of sequential steps where each step does some operation on an internal stream of items.

Here we only present the steps that we need for this task. For a list of all steps and associated documentation, see https://squirro.github.io/nlp/.

First we instantiate an empty pipeline:

In [ ]:
workflow["pipeline"] = []

#### Loader step

The first step is to load the data from Squirro into libNLP and convert them to libNLP's internal format. This step will be passed the various `dataset` settings we gave above since it is the beginning of the pipeline.

In [ ]:
workflow['pipeline'].append({
    "step": "loader",
    "type": "squirro_query",
    "fields": ["body", "title", "keywords.label"]
})

Notice that we specified the `fields` we wanted to import to make loading more efficient.

Also note, that when the loader step gets content, it will always turn it into a flat dictionary before passing it to the next step in the pipeline. This is why we prepend `keywords.` to the fields.

#### Filter steps

Next we want to filter down our dataset, and adjust its format.

In [ ]:
# remove empty entries
workflow['pipeline'].append({
    "step": "filter",
    "type": "empty",
    "fields": ["title", "body"]
})

# merge title and body into 1 field
workflow['pipeline'].append({
    "step": "filter",
    "type": "merge",
    "input_fields": ["title", "body"],
    "output_field": "text"
})

With these 2 filters we have removed all items that are missing `title` or `body`, and merged `body` and `title` into a single field called `text`.

#### Normalization step

We next need to normalize the incoming data so that all the training samples are in the same format. This makes training the model simpler since it shrinks the space of data it has to be able to predict on.

In [ ]:
workflow['pipeline'] += [{
    "step": "normalizer",
    "type": "html",
    "fields": ["text"]
},{
    "step": "normalizer",
    "type": "character",
    "fields": ["text"]
},{
    "step": "normalizer",
    "type": "punctuation",
    "fields": ["text"]
},{
    "step": "normalizer",
    "type": "lowercase",
    "fields": ["text"]
},{
    "step": "normalizer",
    "type": "stopwords",
    "fields": ["text"],
    "stopwords": ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "b", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "c", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "gt", "had", "has", "have", "having", "he", "hed", "hell", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how", "hows", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "it", "its", "its", "itself", "lets", "me", "more", "most", "my", "myself", "nor", "not", "o", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "q", "re", "s", "same", "she", "shed", "shell", "shes", "should", "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then", "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through", "to", "too", "under", "until", "up", "v", "very", "was", "we", "wed", "well", "were", "weve", "were", "what", "whats", "when", "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why", "whys", "with", "would", "x", "you", "youd", "youll", "youre", "youve", "your", "yours", "yourself", "yourselves"]
}]

Here for the field `body`, we are first stripping out `html`, numeric `character`s, and `punctuation`, and then making everything `lowercase`. Finally, we remove `stopwords` for which we provided a list of words.

#### Tokenization step

Now we need to split our input from a stream of words into a list of tokens. For this particular case, we can use the `spaces` tokenizer to get our a sequential list of words.

In [ ]:
workflow['pipeline'].append({
    "step": "tokenizer",
    "type": "spaces",
    "fields": ["text"]
})

#### Embedding step

Right before classification, we have to convert our list of tokenized words into numbers. This is done via an `embedder` step. Squirro comes shipped with some pre-trained embeddings, but for this case, we'll make our own TF-IDF embeddings.

In [ ]:
workflow['pipeline'].append({
    "step": "embedder",
    "type": "tfidf",
    "input_field": "text",
    "output_field": "embedded_text"
})

#### Projection step

The previous step produces very large vectors (essentially the size is the number of unique words in your corpus). To reduce this space, it can be helpful to do a projection down to a smaller vector space. That is exactly what a projection step accomplishes.

In [ ]:
workflow['pipeline'].append({
    "step": "projector",
    "type": "sklearn",
    "model_type": "svd",
    "n_components": 100,
    "input_field": "embedded_text",
    "output_field": "embedded_text"
})

Here we've chosen to reduce the vector size to 100 by using an `svd` projector from scikit-learn.

#### Clustering step

We are now ready to cluster the incoming items (which are now represented as numerical vectors). For this task we'll use a Gaussian Mixture Model clusterer from scikit-learn.

In [ ]:
workflow['pipeline'].append({
    "step": "clusterer",
    "type": "gmm",
    "n_clusters": 3,
    "input_field": "embedded_text",
    "output_field": "keywords.topics",
    "explanation_field": "keywords.sig_terms",
    "term_field": "text"
})

This clusterer takes our input field `embedded_text` and attempts to predict the cluster all the items into up to 3 clusters. We will write the name of these clusters to `keywords.topics`. Also on each item, we'll provide a list of significant terms in `keywords.sig_terms`. These make up the core of each topic.

#### Saver step

Finally we want to save our predictions back to Squirro. We do this through a saver step:

In [ ]:
workflow["pipeline"].append({
    "step": "saver",
    "type": "squirro_item",
    "fields": ["keywords.topics", "keywords.sig_terms"]
})

Note that only the fields we specify in `fields` will be sent back to Squirro.

### All together

Putting it all together, our libNLP workflow looks like this:

In [ ]:
import json
print json.dumps(workflow, indent=2)

## Train the model

Now we're ready to train our proposed workflow. To do that we can simply push it to the Squirro Machine Learning Service:

In [ ]:
ml_workflow_id = client.new_machinelearning_workflow(
    PROJECT_ID, name='20_newsgroups', config=workflow).get('id')
print ml_workflow_id

Now we create a training job for the workflow. This will tell the Machine Learning Service to schedule a job that runs the workflow with the `train` dataset we specified above.

In [ ]:
training_job_id = client.new_machinelearning_job(
    PROJECT_ID, ml_workflow_id=ml_workflow_id, type='training').get('id')
print training_job_id

Now we just wait for it to finish. Depending on the size the dataset, size of the model, and the number of free parameters, this can take anywhere from a few seconds to days. Because of this, it's always a good idea to START SMALL with a test dataset and model until you're confident things are working well.

Since training will take up to 5 minutes to finish, we write the simple function below that pings the job status every 5 seconds. Once this cell is done evaluating, we'll be ready to move on.

In [ ]:
import time
def wait_for_ml_job(project_id, ml_workflow_id, ml_job_id, max_wait_time=600):
    """Wait for ML job to finish"""
    start_time = time.time()
    while True:
        job = client.get_machinelearning_job(
            project_id, ml_workflow_id, ml_job_id, include_run_log=True).get('machinelearning_job')
        if job.get('last_error_at') is not None or job.get('last_success_at') is not None:
            print job.get('logs')
            break
        else:
            print '.',
            time.sleep(5)
        if (time.time() - start_time) > max_wait_time:
            print 'max_wait_time has been exceeded!'
            print job.get('logs')
            break
wait_for_ml_job(PROJECT_ID, ml_workflow_id, training_job_id, max_wait_time=300)

## Analyze the model quality

Now that our model is trained, we can check out how it performed on our test data set (again in this instance it was the same as the training set).

In [ ]:
result = client.get_machinelearning_job(
    PROJECT_ID, ml_workflow_id, training_job_id).get('machinelearning_job').get('last_result')
print json.dumps(result, indent=2)

The above results tell us several things. First `tag_counts` shows us how each ground truth label corresponds to different generate topics. We can see for each label, there is one topic with the majority of the spectral weight. The title for the topic has been generated as well, taking significant terms from a TF-IDF calculation.

Also presented are 3 scores. The `homogeneity_score` measures for each topic, the fraction of the majority label and averages them together. The `completeness_score` measures for each label, the fraction that belongs to the majority topic and averages them together. Finally the `adjusted_rand_score` is a measure of the dissimilarity of the generated topics.

## Validate the model on new data

Since our model has reasonble (though not perfect) quality, we can now move on to validating it on samples that don't yet have a `label`. We can do this in a few different ways, which we cover below.

### Direct inference

First, it's good to do a sanity check. The simplest way to check our model on new data is to run a direct inference on items we have.

In [ ]:
test_items = [{"id": 0, "title": "God is dead", "body": "There is no god, I'm an atheist."},
              {"id": 1, "title": "Jesus saves", "body": "Jesus washes away all our sins."},
              {"id": 2, "title": "Video games for sale", "body": "Ocarina of time is the greatest game ever made, and I'm selling the gold catridge version!"}]

test_items_pred = client.run_machinelearning_workflow(
    PROJECT_ID, ml_workflow_id, data={'items': test_items})['items']
for item, item_pred in zip(test_items, test_items_pred):
    print u'{topics} - {title}'.format(title=item['title'], topics=item_pred['keywords']['topics'])

Seems reasonable...

### Add a pipelet to for ingestion

Now that we have some confidence in our trained model, we can set up a pipelet step that will run items through it during ingestion. For this we have made an example pipelet here: https://github.com/squirro/delivery/tree/master/templates/pipelets/machinelearning.

### Add an inference job for future data

If we want to avoid blocking the ingestion process, we can instead make an ayschronous inference job that will tag new items with our trained model

In [ ]:
inference_job_id = client.new_machinelearning_job(
    PROJECT_ID, ml_workflow_id=ml_workflow_id, type='inference').get('id')
print inference_job_id

## Reset

WARNING: This deletes the project!!!

In [ ]:
client.delete_project(PROJECT_ID)